# LangGraph Agent Patterns - Building Effective Agents

This colab demonstrates common agent patterns from "Building effective agents" using the LangGraph framework. We'll implement several key patterns:
1. Memory-based Agent
2. Tool-using Agent  
3. Multi-agent Collaboration
4. Planning Agent

We'll use LangGraph's workflow system to build these patterns and integrate with LangSmith for tracing and debugging. For this demo, we'll use the free Gemini API from Google.

In [1]:
# Install required dependencies
!pip install -q langgraph langsmith langchain langchain-google-genai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
# Import necessary libraries
import os
import json
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize LLM with Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=GOOGLE_API_KEY
)

## 1. Memory-based Agent Implementation

This agent maintains conversation history and context across interactions. It uses LangGraph's checkpointing mechanism to store state between calls, enabling the agent to remember previous interactions and provide contextual responses.

Key features:
- Persistent memory across sessions
- Context-aware responses
- State management with MemorySaver
- Uses free Gemini API

In [ ]:
# Create a simple memory-based agent
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict
import operator

# Define the state for our agent
class AgentState(TypedDict):
   messages: Annotated[list, operator.add]

# Create a simple agent that remembers context
def memory_agent(state: AgentState):
   # Get the last message
   last_message = state["messages"][-1]

   # Convert messages to string format for context
   context_str = "\n".join([f"{msg.__class__.__name__}: {msg.content}" for msg in state["messages"]])

   # Generate a response based on all previous messages
   prompt = f"Context of conversation:\n{context_str}\n\nCurrent message: {last_message.content}\n\nResponse:"
   response = llm.invoke(prompt)

   return {"messages": [AIMessage(content=response.content)]}

# Create the graph
memory_graph = StateGraph(AgentState)
memory_graph.add_node("agent", memory_agent)
memory_graph.set_entry_point("agent")
memory_graph.add_edge("agent", END)

# Compile with checkpointer for memory
memory_checkpointer = MemorySaver()
memory_compiled = memory_graph.compile(checkpointer=memory_checkpointer)

# Test the memory agent
config = {"configurable": {"thread_id": "test_thread"}}
result = memory_compiled.invoke(
   {"messages": [HumanMessage(content="My favorite color is blue")]},
   config=config
)
print("Agent:", result["messages"][-1].content)

# Test that it remembers
result2 = memory_compiled.invoke(
   {"messages": [HumanMessage(content="What's my favorite color?")]},
   config=config
)
print("Agent:", result2["messages"][-1].content)

Agent: That's a great choice! Blue is a very popular and versatile color. What do you like about blue?
Agent: You told me earlier that your favorite color is blue!


## 2. Tool-using Agent

This pattern demonstrates an agent that can use external tools to perform tasks. The agent uses Wikipedia as an example tool to retrieve information when needed.

Key features:
- Tool integration with LangGraph
- React-style reasoning (Thought-Action-Observation)
- Dynamic tool selection based on user queries
- Free Wikipedia API integration

In [4]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=cac038d4b9855a3fa42d646a400499f737093daa5426209fb40f00003bb8290a
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [6]:
from typing import TypedDict, Annotated
import operator

# Create Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Define tool-using agent state
class ToolState(TypedDict):
   messages: Annotated[list, operator.add]

def tool_agent(state: ToolState):
   messages = state["messages"]
   last_message = messages[-1]

   # Create a simple tool-using agent
   if "search" in last_message.content.lower() or "find" in last_message.content.lower():
       # Extract search query
       search_query = last_message.content.replace("search for", "").replace("find", "").strip()

       # Use Wikipedia tool
       search_result = wikipedia.run(search_query)

       response = f"I searched for '{search_query}' and found:\n\n{search_result}"
   else:
       # Regular response
       response = llm.invoke(last_message.content)
       response = response.content

   return {"messages": [AIMessage(content=response)]}

# Create tool agent graph
tool_graph = StateGraph(ToolState)
tool_graph.add_node("agent", tool_agent)
tool_graph.set_entry_point("agent")
tool_graph.add_edge("agent", END)

# Compile and test
tool_compiled = tool_graph.compile()

# Test tool usage
result = tool_compiled.invoke({"messages": [HumanMessage(content="search for Newton")]})
print("Tool Agent:", result["messages"][-1].content)

# Test normal conversation
result2 = tool_compiled.invoke({"messages": [HumanMessage(content="Tell me a joke")]})
print("Tool Agent:", result2["messages"][-1].content)

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Tool Agent: I searched for 'Newton' and found:

Page: Isaac Newton
Summary: Sir Isaac Newton (; 4 January [O.S. 25 December] 1643 – 31 March [O.S. 20 March] 1727) was an English polymath active as a mathematician, physicist, astronomer, alchemist, theologian, and author. Newton was a key figure in the Scientific Revolution and the Enlightenment that followed. His book Philosophiæ Naturalis Principia Mathematica (Mathematical Principles of Natural Philosophy), first published in 1687, achieved the first great unification in physics and established classical mechanics. Newton also made seminal contributions to optics, and shares credit with German mathematician Gottfried Wilhelm Leibniz for formulating infinitesimal calculus, though he developed calculus years before Leibniz. Newton contributed to and refined the scientific method, and his work is considered the most influential in bringing forth modern science.
In the Principia, Newton formulated the laws of motion and universal gravita

## 3. Multi-agent Collaboration

This pattern shows how multiple specialized agents can work together on a task. Each agent has a specific role and they pass information between themselves to complete complex tasks.

Key features:
- Multiple specialized agents
- Agent collaboration workflow
- State sharing between agents
- Task delegation and coordination

In [7]:
# Define multi-agent state
class MultiAgentState(TypedDict):
   messages: Annotated[list, operator.add]
   task_type: str
   current_agent: str
   result: str

# Create specialized agents
def researcher_agent(state: MultiAgentState):
   """Researches information using Wikipedia"""
   task = state["messages"][-1].content

   if "research" in task.lower() or "find information" in task.lower():
       search_query = task.split("about")[-1].strip() if "about" in task else task
       research_result = wikipedia.run(search_query)

       response = f"Research findings: {research_result}"
       return {"messages": [AIMessage(content=response)], "current_agent": "writer", "result": research_result}

   return {"messages": [AIMessage(content="No research task identified")], "current_agent": "done"}

def writer_agent(state: MultiAgentState):
   """Writes content based on research"""
   research_result = state.get("result", "")

   if research_result:
       prompt = f"Based on this research: {research_result}\n\nWrite a brief summary:"
       response = llm.invoke(prompt)
       return {"messages": [AIMessage(content=response.content)], "current_agent": "done"}

   return {"messages": [AIMessage(content="No research data to write from")], "current_agent": "done"}

# Create multi-agent workflow
multi_agent_graph = StateGraph(MultiAgentState)
multi_agent_graph.add_node("researcher", researcher_agent)
multi_agent_graph.add_node("writer", writer_agent)

# Set entry point
multi_agent_graph.set_entry_point("researcher")

# Add conditional edges
def router(state: MultiAgentState):
   if state["current_agent"] == "writer":
       return "writer"
   else:
       return END

multi_agent_graph.add_conditional_edges("researcher", router, {"writer": "writer", END: END})
multi_agent_graph.add_edge("writer", END)

# Compile and test
multi_agent = multi_agent_graph.compile()

# Test multi-agent collaboration
result = multi_agent.invoke({
   "messages": [HumanMessage(content="research about quantum computing")],
   "task_type": "research_and_write",
   "current_agent": "researcher",
   "result": ""
})

print("Multi-Agent Result:")
for msg in result["messages"]:
   print(f"{msg.__class__.__name__}: {msg.content}")

Multi-Agent Result:
HumanMessage: research about quantum computing
AIMessage: Research findings: Page: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing takes advantage of this behavior using specialized hardware. Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any modern "classical" computer. Theoretically a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing physical simulations; however, the current state of the art is largely experimental and impractical, with several obstacles to useful applications.
The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike 

## 4. Planning Agent

This pattern demonstrates an agent that can break down complex tasks into steps and execute them systematically. The planning agent creates a plan of action and then executes each step sequentially.

Key features:
- Task decomposition
- Sequential execution
- Step-by-step tracking
- Goal-oriented behavior

In [8]:
# Define planning agent state
class PlanningState(TypedDict):
   messages: Annotated[list, operator.add]
   plan: list
   current_step: int
   step_results: list

In [9]:
def planning_agent(state: PlanningState):
   """Creates a plan for complex tasks"""
   task = state["messages"][-1].content

   # Generate a plan
   prompt = f"""Break down this task into 3-5 simple steps: {task}

Format your response as a numbered list:
1. [step]
2. [step]
3. [step]"""

   plan_response = llm.invoke(prompt)

   # Extract plan steps
   plan_text = plan_response.content
   steps = []
   for line in plan_text.split('\n'):
       if line.strip() and any(line.strip().startswith(str(i)) for i in range(1, 6)):
           steps.append(line.strip())

   return {
       "messages": [AIMessage(content=f"Created plan:\n{plan_text}")],
       "plan": steps,
       "current_step": 0,
       "step_results": []
   }

In [10]:
def execute_step(state: PlanningState):
   """Executes the current step in the plan"""
   if state["current_step"] >= len(state["plan"]):
       final_summary = "\n".join(state["step_results"])
       return {
           "messages": [AIMessage(content=f"Plan completed. Results:\n{final_summary}")],
           "current_step": state["current_step"]
       }

   current_step = state["plan"][state["current_step"]]

   # Execute current step
   prompt = f"Execute this step: {current_step}\nProvide a brief result."
   result = llm.invoke(prompt)

   return {
       "messages": [AIMessage(content=f"Executed: {current_step}\nResult: {result.content}")],
       "current_step": state["current_step"] + 1,
       "step_results": state["step_results"] + [f"Step {state['current_step'] + 1}: {result.content}"]
   }

# Create planning agent workflow
planning_graph = StateGraph(PlanningState)
planning_graph.add_node("planner", planning_agent)
planning_graph.add_node("executor", execute_step)

planning_graph.set_entry_point("planner")
planning_graph.add_edge("planner", "executor")

# Add loop for executing all steps
def should_continue(state: PlanningState):
   if state["current_step"] >= len(state["plan"]):
       return END
   return "executor"

planning_graph.add_conditional_edges("executor", should_continue, {"executor": "executor", END: END})

# Compile and test
planning_compiled = planning_graph.compile()

# Test planning agent
result = planning_compiled.invoke({
   "messages": [HumanMessage(content="Plan and execute: Create a simple chatbot")],
   "plan": [],
   "current_step": 0,
   "step_results": []
})

print("Planning Agent Results:")
for msg in result["messages"]:
   print(f"\n{msg.__class__.__name__}: {msg.content}")

Planning Agent Results:

HumanMessage: Plan and execute: Create a simple chatbot

AIMessage: Created plan:
Here's a breakdown of planning and executing a simple chatbot:

1. **Define the Chatbot's Purpose & Scope:** Decide what your chatbot will *do*. Will it answer FAQs, schedule appointments, provide basic information, or something else? Keep it very narrow and simple for a first project.  Determine the key questions/topics it needs to handle.

2. **Choose a Platform/Tool & Build the Basic Structure:** Select a chatbot platform (e.g., Dialogflow, Rasa, Microsoft Bot Framework, or even a simple Python script).  Create the basic structure:  define intents (user goals), entities (key information), and responses for the most common user inputs.

3. **Train & Test Your Chatbot:**  Provide example phrases for each intent and test the chatbot with different inputs. Refine the responses and add more training data to improve accuracy and handle edge cases.

AIMessage: Executed: 1. **Define th

## Summary and Testing

Let's create a simple test dashboard to demonstrate all agent patterns and visualize their execution flow:

- Memory-based Agent: Maintains conversation context
- Tool-using Agent: Integrates external APIs
- Multi-agent Collaboration: Coordinates multiple specialized agents
- Planning Agent: Breaks down and executes complex tasks

This completes the LangGraph implementation. For the CrewAI implementation, we'll need to structure the patterns differently using CrewAI's framework.

In [12]:
# Create a simple memory-based agent
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict
import operator

# Define the state for our agent
class AgentState(TypedDict):
   messages: Annotated[list, operator.add]

# Create a simple agent that remembers context
def memory_agent(state: AgentState):
   # Get the last message
   last_message = state["messages"][-1]

   # Convert messages to string format for context
   context_str = "\n".join([f"{msg.__class__.__name__}: {msg.content}" for msg in state["messages"]])

   # Generate a response based on all previous messages
   prompt = f"Context of conversation:\n{context_str}\n\nCurrent message: {last_message.content}\n\nResponse:"
   response = llm.invoke(prompt)

   return {"messages": [AIMessage(content=response.content)]}

# Create the graph
memory_graph = StateGraph(AgentState)
memory_graph.add_node("agent", memory_agent)
memory_graph.set_entry_point("agent")
memory_graph.add_edge("agent", END)

# Compile with checkpointer for memory
memory_checkpointer = MemorySaver()
memory_compiled = memory_graph.compile(checkpointer=memory_checkpointer)

# Test the memory agent
config = {"configurable": {"thread_id": "test_thread"}}
result = memory_compiled.invoke(
   {"messages": [HumanMessage(content="My favorite color is blue")]},
   config=config
)
print("Agent:", result["messages"][-1].content)

# Test that it remembers
result2 = memory_compiled.invoke(
   {"messages": [HumanMessage(content="What's my favorite color?")]},
   config=config
)
print("Agent:", result2["messages"][-1].content)

Agent: That's cool! What shade of blue do you like best?
Agent: You told me your favorite color is blue.


In [13]:
# Create a test dashboard to demonstrate all patterns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Create visualization of agent patterns
fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(2, 2, figure=fig)

# Test all patterns and visualize
patterns = {
   "Memory Agent": memory_compiled,
   "Tool Agent": tool_compiled,
   "Multi-Agent": multi_agent,
   "Planning Agent": planning_compiled
}

test_queries = {
   "Memory Agent": ["My name is Alice", "What's my name?"],
   "Tool Agent": ["search for Python programming"],
   "Multi-Agent": ["research about machine learning"],
   "Planning Agent": ["Plan: write a blog post about AI"]
}

results = {}

<Figure size 1500x1000 with 0 Axes>

In [14]:


# Test each pattern
for pattern_name, agent in patterns.items():
   if pattern_name == "Memory Agent":
       config = {"configurable": {"thread_id": "demo_thread"}}
       result1 = agent.invoke({"messages": [HumanMessage(content=test_queries[pattern_name][0])]}, config=config)
       result2 = agent.invoke({"messages": [HumanMessage(content=test_queries[pattern_name][1])]}, config=config)
       results[pattern_name] = [result1["messages"][-1].content, result2["messages"][-1].content]
   else:
       if pattern_name == "Multi-Agent":
           result = agent.invoke({
               "messages": [HumanMessage(content=test_queries[pattern_name][0])],
               "task_type": "research_and_write",
               "current_agent": "researcher",
               "result": ""
           })
           results[pattern_name] = [msg.content for msg in result["messages"] if isinstance(msg, AIMessage)]
       elif pattern_name == "Planning Agent":
           result = agent.invoke({
               "messages": [HumanMessage(content=test_queries[pattern_name][0])],
               "plan": [],
               "current_step": 0,
               "step_results": []
           })
           results[pattern_name] = [msg.content for msg in result["messages"] if isinstance(msg, AIMessage)]
       else:
           result = agent.invoke({"messages": [HumanMessage(content=test_queries[pattern_name][0])]})
           results[pattern_name] = [result["messages"][-1].content]

# Display results
for i, (pattern_name, pattern_results) in enumerate(results.items()):
   ax = fig.add_subplot(gs[i // 2, i % 2])
   ax.text(0.5, 0.9, pattern_name, ha='center', va='center', fontsize=14, fontweight='bold')

   result_text = "\n".join([f"Response: {r[:100]}..." if len(r) > 100 else f"Response: {r}" for r in pattern_results])
   ax.text(0.5, 0.5, result_text, ha='center', va='center', fontsize=10, wrap=True)
   ax.set_xlim(0, 1)
   ax.set_ylim(0, 1)
   ax.axis('off')

plt.tight_layout()
plt.show()

# Print summary
print("\n=== LangGraph Agent Patterns Summary ===")
for pattern_name, pattern_results in results.items():
   print(f"\n{pattern_name}:")
   for j, result in enumerate(pattern_results):
       print(f"  Response {j+1}: {result[:200]}...")

<Figure size 640x480 with 0 Axes>


=== LangGraph Agent Patterns Summary ===

Memory Agent:
  Response 1: Okay, it seems like you're repeating yourself. Is there anything else I can help you with, Alice?...
  Response 2: You told me your name is Alice....

Tool Agent:
  Response 1: I searched for 'Python programming' and found:

Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readabi...

Multi-Agent:
  Response 1: Research findings: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can lear...
  Response 2: Machine learning (ML) is a field of AI where statistical algorithms learn from data to perform tasks without explicit instructions. Deep learning, a subfield of ML, utilizes neural networks to achieve...

Planning Agent:
  Response 1: Created plan:
Here's a breakdown of planning a blog post about AI i